<a href="https://colab.research.google.com/github/KyoungmiKwon/Bigdata_Training_at_ITwill/blob/main/Team_Project/C03_KM_Mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mecab Colab 설치하기

In [2]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
%env JAVA_HOME "/user/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/user/lib/jvm/java-8-openjdk-amd64"


In [5]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141807 sha256=1b05f715908826dd847bd9454b3f51d9f26af46c79a732f85cb30cec1527be72
  Stored in directory: /root/.cache/pip/wheels/99/75/a6/e9e73a1dbd73579383644942ef18a6d17ad728a3052a1147fb
Successfully built mecab-python
  Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2


#import

In [6]:
import konlpy
from konlpy.tag import Mecab

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer # 토큰화
import pandas as pd                                       # 파일읽기
import numpy as np                                        #
import matplotlib.pyplot as plt                           # 그래프

#뉴스 기사 데이터 불러오기

In [8]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Project/C01_KM_Data_LabelingResult.xlsx',sheet_name='Sheet2',header=0)

In [9]:
data[:2]

,No,Code,Data,Journal,Title,Text,URL,Sample Y/N,In Charge,-1 / 0 / 1
0,52,5930,2021.04.13 17:50,파이낸셜뉴스,"삼성전자 사내식당 2곳, 신세계·풀무원이 운영한다",삼성전자가 사내식당 2곳에 대해 실시한 외부 급식업체 경쟁입찰에서 '신세계푸드'와 ...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
1,87,5930,2021.04.13 03:11,파이낸셜뉴스,"삼성전자 초청, 美 '반도체 회담'‥백악관 ""즉각적 결정 없을 것""",조 바이든 미국 대통령이 12일(현지시간) '반도체 공급망 강화 온라인 회담'에 참...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,-1


In [10]:
news = data[:10]
news

,No,Code,Data,Journal,Title,Text,URL,Sample Y/N,In Charge,-1 / 0 / 1
0,52,5930,2021.04.13 17:50,파이낸셜뉴스,"삼성전자 사내식당 2곳, 신세계·풀무원이 운영한다",삼성전자가 사내식당 2곳에 대해 실시한 외부 급식업체 경쟁입찰에서 '신세계푸드'와 ...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
1,87,5930,2021.04.13 03:11,파이낸셜뉴스,"삼성전자 초청, 美 '반도체 회담'‥백악관 ""즉각적 결정 없을 것""",조 바이든 미국 대통령이 12일(현지시간) '반도체 공급망 강화 온라인 회담'에 참...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,-1
2,190,5930,2021.04.08 14:55,파이낸셜뉴스,"[특징주]로보스타, 삼성-LG전자 미래 신성장 동력 ‘로봇산업’ 육성 ...",[파이낸셜뉴스]미래 신성장 동력인 로봇산업에 삼성전자와 LG전자가 힘을 쏟을 것이란...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
3,207,5930,2021.04.07 20:41,파이낸셜뉴스,삼성전자 1분기'역대급 성적표',"영업익 전망 웃도는 9조3000억LG전자, 1조5178억 사상 최대 삼성전자와 L...",https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
4,233,5930,2021.04.07 15:04,조선비즈,"삼성전자, 1분기 깜짝 실적에도 주가 하락",삼성전자(005930)가 올해 1분기 깜짝 실적을 기록했는데도 주가가 하락하고 있다...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,-1
5,243,5930,2021.04.07 09:26,파이낸셜뉴스,"삼성운용, ‘2021 내 마음속 원픽 펀드’ 이벤트 진행",2021년 베스트펀드 운용 펀드매니저가 직접 유튜브에 출연 [파이낸셜뉴스] 삼성...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
6,357,5930,2021.04.05 18:33,파이낸셜뉴스,"‘삼성 독주 체제’ 선택권 축소 우려 ""단말기 가격 오르나"" 소비자 부...",국내 휴대폰 시장 영향은 LG전자가 스마트폰 사업에서 철수를 결정하면서 국내 스마...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,-1
7,360,5930,2021.04.05 14:00,파이낸셜뉴스,"삼성전자, 올해도 기초과학 분야에 464억 '통 큰 지원'",2021년 상반기 삼성미래기술육성사업 지원과제에 선정된 (왼쪽부터)서울대학교 류경석...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
8,378,5930,2021.04.05 11:00,파이낸셜뉴스,"관리의 삼성', 차세대 경영혁신 플랫폼 N-ERP 도입",삼성전자의 새로운 비지니스 플랫폼인 'N-ERP'의 로고 [파이낸셜뉴스] 삼성전자...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
9,400,5930,2021.04.05 06:03,조선비즈,[주간증시전망] 삼성전자와 미국의 성적표를 살펴보자,지난주(3월29일~4월2일) 코스피지수는 3025.39(29일 최저치)~3121.5...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,0


# 텍스트 정제

In [11]:
mc = Mecab() # 품사태그 사전

''' # Mecab Method #
Mecab.nouns()  # 명사 추출
Mecab.morphs() # 형태소 추출
Mecab.pos()    # 품사 부착
'''

' # Mecab Method #\nMecab.nouns()  # 명사 추출\nMecab.morphs() # 형태소 추출\nMecab.pos()    # 품사 부착\n'

## 1) 한글 제외한 문자 제거

In [12]:
# 기사에서 한글 제외하고 모두 제거
news['Text'] = news['Text'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 2) 특정 품사만 추출

[Mecab 품사사전](https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=589544265)

In [13]:
# 1) 기사에서 특정 품사 추출 (기사별)

tag_cls = ['NNG','VV+EC','XSV+EP', 'SL']
# NNG = 일반명사, VV = 동사, EC = 연결어미, XSV = 동사파생접미사, EP = 선어말 어미, SL = 외국어

artc = []
for line in news['Text'][:]:
    temp = []  
    for word in mc.pos(line):
        if word[1] in tag_cls:
            temp.append(word[0])
    artc.append(temp)

print(artc[:])
print(len(artc))

[['사내', '식당', '곳', '대해', '실시', '외부', '급식', '업체', '경쟁', '입찰', '신세계', '푸드', '푸드', '컬처', '운영', '업체', '선정', '됐', '일', '월', '단체', '급식', '다양', '메뉴', '향상', '서비스', '통해', '임직원', '복리', '후생', '향상', '사업장', '기흥', '사업장', '사내', '식당', '곳', '대해', '공개', '입찰', '공고', '했', '업체', '참여', '이번', '입찰', '서류', '심사', '거쳐', '메뉴', '구성', '서비스', '평가', '프레젠테이션', '업체', '인프라', '위생', '점검', '현장', '실사', '임직원', '음식', '품평회', '단계', '공정', '절차', '통해', '진행', '됐', '기존', '단체', '급식', '공급', '했', '스토리', '입찰', '배제', '됐', '최종', '선정', '신세계', '푸드', '푸드', '컬처', '반', '동안', '고용', '승계', '업무', '인수', '후', '월', '일', '운영', '시작', '예정', '관계자', '신규', '업체', '임직원', '만족도', '운영', '보완', '점', '검토', '후', '사내', '식당', '대해서', '경쟁', '입찰', '확대', '계획', '말', '했', '기자', '저작', '뉴스', '무단', '전재', '배포', '금지', '뉴스'], ['대통령', '현지', '시간', '반도체', '공급', '망', '강화', '온라인', '회담', '참석', '인사말', '실시간', '중계', '뉴스', '백악관', '세계', '기업', '불러', '반도체', '공급', '부족', '대해', '기업', '의견', '가운데', '이번', '회담', '결정', '발표', '백악관', '대변인', '현지', '시간', '정례', '브리핑', '대통령', '회의', '참석', '이유', '기업', '의견',

In [57]:
# 2) 기사에서 특정품사 추출 (모든 기사에서) -- 

tag_cls = ['NNG','VV+EC','XSV+EP', 'SL']
# NNG = 일반명사, VV = 동사, EC = 연결어미, XSV = 동사파생접미사, EP = 선어말 어미, SL = 외국어

voca = []
for line in news['Text'][:]:
    temp = []  
    for word in mc.pos(line):
        if word[1] in tag_cls:
            voca.append(word[0])

print(voca[:])
print(len(voca))

['사내', '식당', '곳', '대해', '실시', '외부', '급식', '업체', '경쟁', '입찰', '신세계', '푸드', '푸드', '컬처', '운영', '업체', '선정', '됐', '일', '월', '단체', '급식', '다양', '메뉴', '향상', '서비스', '통해', '임직원', '복리', '후생', '향상', '사업장', '기흥', '사업장', '사내', '식당', '곳', '대해', '공개', '입찰', '공고', '했', '업체', '참여', '이번', '입찰', '서류', '심사', '거쳐', '메뉴', '구성', '서비스', '평가', '프레젠테이션', '업체', '인프라', '위생', '점검', '현장', '실사', '임직원', '음식', '품평회', '단계', '공정', '절차', '통해', '진행', '됐', '기존', '단체', '급식', '공급', '했', '스토리', '입찰', '배제', '됐', '최종', '선정', '신세계', '푸드', '푸드', '컬처', '반', '동안', '고용', '승계', '업무', '인수', '후', '월', '일', '운영', '시작', '예정', '관계자', '신규', '업체', '임직원', '만족도', '운영', '보완', '점', '검토', '후', '사내', '식당', '대해서', '경쟁', '입찰', '확대', '계획', '말', '했', '기자', '저작', '뉴스', '무단', '전재', '배포', '금지', '뉴스', '대통령', '현지', '시간', '반도체', '공급', '망', '강화', '온라인', '회담', '참석', '인사말', '실시간', '중계', '뉴스', '백악관', '세계', '기업', '불러', '반도체', '공급', '부족', '대해', '기업', '의견', '가운데', '이번', '회담', '결정', '발표', '백악관', '대변인', '현지', '시간', '정례', '브리핑', '대통령', '회의', '참석', '이유', '기업', '의견', '문

## 3) 불용어처리
        불용어 list 생성 -> 불용어 제외 한 단어만 append

In [58]:
swords = ['삼성전자','삼성','기자','인텔','신세계','풀무원','원','픽','내','마음속','억','통','곳','것','파이낸셜','뉴스','억','명','김','말','간','조선','비즈','조선비즈',
          '저작','무단','전재','배포','금지','월일']

X_ts = [] # 신문 기사별, 단어 저장할 리스트

for word in voca:            # 단어를 사전에서 꺼내서
    if word not in swords:   # 불용어 리스트에 단어가 없다면
        X_ts.append(word)    # X_ts에 추가

len(X_ts)

1848

In [27]:
X_ts[:5]

['사내', '식당', '대해', '실시', '외부']

## 4) 길이가 짧은 글자 제거 (optional)

In [59]:
X_ts_=[]
for word in X_ts:
    if len(word) > 1:
        X_ts_.append(word)
print(X_ts_)

['사내', '식당', '대해', '실시', '외부', '급식', '업체', '경쟁', '입찰', '푸드', '푸드', '컬처', '운영', '업체', '선정', '단체', '급식', '다양', '메뉴', '향상', '서비스', '통해', '임직원', '복리', '후생', '향상', '사업장', '기흥', '사업장', '사내', '식당', '대해', '공개', '입찰', '공고', '업체', '참여', '이번', '입찰', '서류', '심사', '거쳐', '메뉴', '구성', '서비스', '평가', '프레젠테이션', '업체', '인프라', '위생', '점검', '현장', '실사', '임직원', '음식', '품평회', '단계', '공정', '절차', '통해', '진행', '기존', '단체', '급식', '공급', '스토리', '입찰', '배제', '최종', '선정', '푸드', '푸드', '컬처', '동안', '고용', '승계', '업무', '인수', '운영', '시작', '예정', '관계자', '신규', '업체', '임직원', '만족도', '운영', '보완', '검토', '사내', '식당', '대해서', '경쟁', '입찰', '확대', '계획', '대통령', '현지', '시간', '반도체', '공급', '강화', '온라인', '회담', '참석', '인사말', '실시간', '중계', '백악관', '세계', '기업', '불러', '반도체', '공급', '부족', '대해', '기업', '의견', '가운데', '이번', '회담', '결정', '발표', '백악관', '대변인', '현지', '시간', '정례', '브리핑', '대통령', '회의', '참석', '이유', '기업', '의견', '문제', '지속', '논의', '일부', '설명', '산업', '협력', '의회', '합의', '이뤄져야', '향후', '부족', '사태', '방지', '국가', '산업', '차원', '합의', '이뤄져야', '이날', '백악관', '국가', '안보', '보좌관', '국가', '경제'

## 5) 낮은 빈도 단어 제거

In [73]:
# step 1) 단어 별 빈도수
## 단어의 빈도 수
vocab ={}
for word in X_ts_:
    if word not in vocab:
        vocab[word] = 0
    vocab[word] += 1
print(vocab)
print(len(vocab))

{'사내': 4, '식당': 3, '대해': 4, '실시': 3, '외부': 2, '급식': 3, '업체': 5, '경쟁': 6, '입찰': 5, '푸드': 4, '컬처': 2, '운영': 3, '선정': 7, '단체': 2, '다양': 1, '메뉴': 2, '향상': 5, '서비스': 2, '통해': 10, '임직원': 5, '복리': 1, '후생': 1, '사업장': 2, '기흥': 1, '공개': 6, '공고': 1, '참여': 3, '이번': 5, '서류': 1, '심사': 1, '거쳐': 1, '구성': 1, '평가': 1, '프레젠테이션': 1, '인프라': 12, '위생': 1, '점검': 2, '현장': 1, '실사': 1, '음식': 1, '품평회': 1, '단계': 1, '공정': 1, '절차': 1, '진행': 2, '기존': 1, '공급': 5, '스토리': 1, '배제': 1, '최종': 2, '동안': 2, '고용': 3, '승계': 1, '업무': 8, '인수': 2, '시작': 1, '예정': 5, '관계자': 2, '신규': 1, '만족도': 1, '보완': 1, '검토': 1, '대해서': 1, '확대': 3, '계획': 7, '대통령': 5, '현지': 7, '시간': 3, '반도체': 14, '강화': 4, '온라인': 3, '회담': 2, '참석': 4, '인사말': 2, '실시간': 3, '중계': 1, '백악관': 5, '세계': 7, '기업': 11, '불러': 1, '부족': 5, '의견': 2, '가운데': 1, '결정': 6, '발표': 16, '대변인': 1, '정례': 1, '브리핑': 1, '회의': 2, '이유': 2, '문제': 3, '지속': 1, '논의': 2, '일부': 1, '설명': 6, '산업': 4, '협력': 1, '의회': 2, '합의': 2, '이뤄져야': 2, '향후': 3, '사태': 2, '방지': 1, '국가': 4, '차원': 1, '이날': 6, '안보': 1, '보좌관': 

In [64]:
# step 2) 빈도수 낮은 단어의 삭제 -- 복잡 버전

# 1. 빈도수가 높은 순서대로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse =True)
## print(vocab_sorted)

# 2. 빈도수가 높은 순서대로 정수 인덱스 부여
word_to_index = {}

i=0
for (word,freq) in vocab_sorted:
    if freq > 1:
        i = i+1
        word_to_index[word] = i
print(word_to_index)

# 3. 빈도수 낮은 단어의 선택
words_del = {}
threshold = 5
for word,cnt in word_to_index.items():
    if cnt < threshold:
        words_del[word]= cnt
print(words_del)

# 4. 빈도 낮은 단어 인덱스 정보 삭제
for w in words_del:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
print(word_to_index)

# 5.Out-Of_Vocabulary(단어 집합에 없는 단어) 추가
word_to_index['OOV'] = len(word_to_index) + 1
print(word_to_index)

{'투자': 1, '시장': 2, '사업': 3, '국내': 4, '발표': 5, '전망': 6, '전자': 7, '반도체': 8, '분기': 9, '실적': 10, '지원': 11, '영업': 12, '기술': 13, '분야': 14, '인프라': 15, '연구': 16, '로봇': 17, '펀드': 18, '교수': 19, '금리': 20, '기업': 21, '과제': 22, '시스템': 23, '물가': 24, '통해': 25, '미래': 26, '소비자': 27, '대비': 28, '부문': 29, '제조사': 30, '상승': 31, '가능': 32, '경기': 33, '업무': 34, '기록': 35, '판매': 36, '지수': 37, '선정': 38, '계획': 39, '현지': 40, '세계': 41, '올해': 42, '이익': 43, '전년': 44, '기준': 45, '증권': 46, '연구원': 47, '경쟁': 48, '공개': 49, '결정': 50, '설명': 51, '이날': 52, '관련': 53, '증가': 54, '수준': 55, '예상': 56, '영향': 57, '필요': 58, '대학교': 59, '과학': 60, '증시': 61, '의사록': 62, '인상': 63, '업체': 64, '입찰': 65, '향상': 66, '임직원': 67, '이번': 68, '공급': 69, '예정': 70, '대통령': 71, '백악관': 72, '부족': 73, '개발': 74, '스타': 75, '철수': 76, '전기': 77, '플랫': 78, '매출': 79, '잠정': 80, '하락': 81, '순매도': 82, '개인': 83, '이벤트': 84, '통사': 85, '위해': 86, '경영': 87, '데이터': 88, '시각': 89, '위원': 90, '사내': 91, '대해': 92, '푸드': 93, '강화': 94, '참석': 95, '산업': 96, '국가': 97, '포함': 98, '글로벌': 99, '해결

In [72]:
# step 2) 빈도수 낮은 단어의 삭제 -- simple 버전

# 1. 빈도수가 높은 순서대로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse =True)
## print(vocab_sorted)

# 2. 빈도수가 높은 순서대로 정수 인덱스 부여
word_to_index = {}

i=0
for (word,freq) in vocab_sorted:
    if freq > 1:
        i = i+1
        word_to_index[word] = i
print(word_to_index)

# 3. 빈도수가 낮은 단어 삭제
word_to_index_1 = {}

threshold = 5
for word,cnt in word_to_index.items():
    if cnt >=  threshold:
        word_to_index_1[word]= cnt
print(word_to_index_1)

# 4.Out-Of_Vocabulary(단어 집합에 없는 단어) 추가
word_to_index_1['OOV'] = len(word_to_index_1) + 1
print(word_to_index_1)
print(len(word_to_index_1))

{'투자': 1, '시장': 2, '사업': 3, '국내': 4, '발표': 5, '전망': 6, '전자': 7, '반도체': 8, '분기': 9, '실적': 10, '지원': 11, '영업': 12, '기술': 13, '분야': 14, '인프라': 15, '연구': 16, '로봇': 17, '펀드': 18, '교수': 19, '금리': 20, '기업': 21, '과제': 22, '시스템': 23, '물가': 24, '통해': 25, '미래': 26, '소비자': 27, '대비': 28, '부문': 29, '제조사': 30, '상승': 31, '가능': 32, '경기': 33, '업무': 34, '기록': 35, '판매': 36, '지수': 37, '선정': 38, '계획': 39, '현지': 40, '세계': 41, '올해': 42, '이익': 43, '전년': 44, '기준': 45, '증권': 46, '연구원': 47, '경쟁': 48, '공개': 49, '결정': 50, '설명': 51, '이날': 52, '관련': 53, '증가': 54, '수준': 55, '예상': 56, '영향': 57, '필요': 58, '대학교': 59, '과학': 60, '증시': 61, '의사록': 62, '인상': 63, '업체': 64, '입찰': 65, '향상': 66, '임직원': 67, '이번': 68, '공급': 69, '예정': 70, '대통령': 71, '백악관': 72, '부족': 73, '개발': 74, '스타': 75, '철수': 76, '전기': 77, '플랫': 78, '매출': 79, '잠정': 80, '하락': 81, '순매도': 82, '개인': 83, '이벤트': 84, '통사': 85, '위해': 86, '경영': 87, '데이터': 88, '시각': 89, '위원': 90, '사내': 91, '대해': 92, '푸드': 93, '강화': 94, '참석': 95, '산업': 96, '국가': 97, '포함': 98, '글로벌': 99, '해결

# 덱스트 정제 using Tokenizer
    위의 텍스트 정제 `1) ~ 4)` 과정은 동일

In [77]:
token = Tokenizer()             # tokenizer 생성
token.fit_on_texts(X_ts_)       # 토큰화
print(token.word_index)         # 단어사전

{'투자': 1, '시장': 2, '사업': 3, '국내': 4, '발표': 5, '전망': 6, '전자': 7, '반도체': 8, '분기': 9, '실적': 10, '지원': 11, '영업': 12, '기술': 13, '분야': 14, '인프라': 15, '연구': 16, '로봇': 17, '펀드': 18, '교수': 19, '금리': 20, '기업': 21, '과제': 22, '시스템': 23, '물가': 24, '통해': 25, '미래': 26, '소비자': 27, '대비': 28, '부문': 29, '제조사': 30, '상승': 31, '가능': 32, '경기': 33, '업무': 34, '기록': 35, '판매': 36, '지수': 37, '선정': 38, '계획': 39, '현지': 40, '세계': 41, '올해': 42, '이익': 43, '전년': 44, '기준': 45, '증권': 46, '연구원': 47, '경쟁': 48, '공개': 49, '결정': 50, '설명': 51, '이날': 52, '관련': 53, '증가': 54, '수준': 55, '예상': 56, '영향': 57, '필요': 58, '대학교': 59, '과학': 60, '증시': 61, '의사록': 62, '인상': 63, '업체': 64, '입찰': 65, '향상': 66, '임직원': 67, '이번': 68, '공급': 69, '예정': 70, '대통령': 71, '백악관': 72, '부족': 73, '개발': 74, '스타': 75, '철수': 76, '전기': 77, '플랫': 78, '매출': 79, '잠정': 80, '하락': 81, '순매도': 82, '개인': 83, '이벤트': 84, '통사': 85, '위해': 86, '경영': 87, '데이터': 88, '시각': 89, '위원': 90, '사내': 91, '대해': 92, '푸드': 93, '강화': 94, '참석': 95, '산업': 96, '국가': 97, '포함': 98, '글로벌': 99, '해결

In [75]:
#> Before <#
print(len(token.word_index)) # 단어 사전의 단어 갯수
print(token.word_counts)
print(token.word_counts.items()) # 단어 별 등장 횟수

678
OrderedDict([('사내', 4), ('식당', 3), ('대해', 4), ('실시', 3), ('외부', 2), ('급식', 3), ('업체', 5), ('경쟁', 6), ('입찰', 5), ('푸드', 4), ('컬처', 2), ('운영', 3), ('선정', 7), ('단체', 2), ('다양', 1), ('메뉴', 2), ('향상', 5), ('서비스', 2), ('통해', 10), ('임직원', 5), ('복리', 1), ('후생', 1), ('사업장', 2), ('기흥', 1), ('공개', 6), ('공고', 1), ('참여', 3), ('이번', 5), ('서류', 1), ('심사', 1), ('거쳐', 1), ('구성', 1), ('평가', 1), ('프레젠테이션', 1), ('인프라', 12), ('위생', 1), ('점검', 2), ('현장', 1), ('실사', 1), ('음식', 1), ('품평회', 1), ('단계', 1), ('공정', 1), ('절차', 1), ('진행', 2), ('기존', 1), ('공급', 5), ('스토리', 1), ('배제', 1), ('최종', 2), ('동안', 2), ('고용', 3), ('승계', 1), ('업무', 8), ('인수', 2), ('시작', 1), ('예정', 5), ('관계자', 2), ('신규', 1), ('만족도', 1), ('보완', 1), ('검토', 1), ('대해서', 1), ('확대', 3), ('계획', 7), ('대통령', 5), ('현지', 7), ('시간', 3), ('반도체', 14), ('강화', 4), ('온라인', 3), ('회담', 2), ('참석', 4), ('인사말', 2), ('실시간', 3), ('중계', 1), ('백악관', 5), ('세계', 7), ('기업', 11), ('불러', 1), ('부족', 5), ('의견', 2), ('가운데', 1), ('결정', 6), ('발표', 16), ('대변인', 1), ('정례', 1), 

## 5) 낮은 빈도 단어 제거

In [81]:
for index,(word,count) in enumerate(token.word_counts.items()):
    print(index,word,count)

0 사내 4
1 식당 3
2 대해 4
3 실시 3
4 외부 2
5 급식 3
6 업체 5
7 경쟁 6
8 입찰 5
9 푸드 4
10 컬처 2
11 운영 3
12 선정 7
13 단체 2
14 다양 1
15 메뉴 2
16 향상 5
17 서비스 2
18 통해 10
19 임직원 5
20 복리 1
21 후생 1
22 사업장 2
23 기흥 1
24 공개 6
25 공고 1
26 참여 3
27 이번 5
28 서류 1
29 심사 1
30 거쳐 1
31 구성 1
32 평가 1
33 프레젠테이션 1
34 인프라 12
35 위생 1
36 점검 2
37 현장 1
38 실사 1
39 음식 1
40 품평회 1
41 단계 1
42 공정 1
43 절차 1
44 진행 2
45 기존 1
46 공급 5
47 스토리 1
48 배제 1
49 최종 2
50 동안 2
51 고용 3
52 승계 1
53 업무 8
54 인수 2
55 시작 1
56 예정 5
57 관계자 2
58 신규 1
59 만족도 1
60 보완 1
61 검토 1
62 대해서 1
63 확대 3
64 계획 7
65 대통령 5
66 현지 7
67 시간 3
68 반도체 14
69 강화 4
70 온라인 3
71 회담 2
72 참석 4
73 인사말 2
74 실시간 3
75 중계 1
76 백악관 5
77 세계 7
78 기업 11
79 불러 1
80 부족 5
81 의견 2
82 가운데 1
83 결정 6
84 발표 16
85 대변인 1
86 정례 1
87 브리핑 1
88 회의 2
89 이유 2
90 문제 3
91 지속 1
92 논의 2
93 일부 1
94 설명 6
95 산업 4
96 협력 1
97 의회 2
98 합의 2
99 이뤄져야 2
100 향후 3
101 사태 2
102 방지 1
103 국가 4
104 차원 1
105 이날 6
106 안보 1
107 보좌관 1
108 경제 2
109 위원회 2
110 위원장 1
111 주재 1
112 해소 1
113 화상 1
114 합류 1
115 일정 1
116 초대 1
117 포함 4
118 구글 1
119 모회사